In [1]:
import os
import pandas as pd
os.chdir('..')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

## 1. Get the sample of parsed AVL data.
Clean MTA prefixes from trip_id

In [2]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/1203.txt',header=None)

# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['ROUTE_ID','latitude','longitude','recorded_time','vehicle_id','TRIP_ID','trip_date','SHAPE_ID',
                   'STOP_ID','distance_stop','distance_shape','status']

bustime.drop_duplicates(['vehicle_id','recorded_time'],inplace=True)
bustime['TRIP_ID'] = bustime['TRIP_ID'].str.replace('MTA NYCT_','')
bustime['TRIP_ID'] = bustime['TRIP_ID'].str.replace('MTABC_','')
bustime.set_index(['ROUTE_ID','TRIP_ID','trip_date','vehicle_id'],inplace=True,drop=True)

# for demonstration, use a subset. Just get data for one trip-date.
tripDateLookup = "2015-12-03" # this is a non-holiday Thursday
bustime = bustime.xs((tripDateLookup),level=(2),drop_level=False)
bustime.sort_index(inplace=True)
bustime['recorded_time'] = bustime['recorded_time'].apply(ttools.parseActualTime,tdate='2015-12-03')
print 'Finished loading BusTime data and and slicing one day.'

Finished loading BusTime data and and slicing one day.


## 2. Convert distance fields to numeric and make a calculated column for "distance along shape"
Also show range of calculated value.

In [3]:
bustime['distance_shape'] = bustime['distance_shape'].convert_objects(convert_numeric=True)
bustime['distance_stop'] = bustime['distance_stop'].convert_objects(convert_numeric=True)
bustime['veh_dist_along_shape'] = bustime['distance_shape'] - bustime['distance_stop']

In [4]:
print (bustime['veh_dist_along_shape'].min(),bustime['veh_dist_along_shape'].max())

(-35705.919999999998, 63991.839999999997)


## 3. Check for multiple vehicles reported on the same trip
Usually there is a clear "winner."  Perhaps this due to logging in/out of vehicles before the trip actually begins.
But sometimes there are many observations recorded from both vehicles.  For further exploration, see <a href='https://github.com/sarangof/Bus-Capstone/blob/master/unstable_trip_ids.ipynb'>this notebook</a>.

In [7]:
# create a GroupBy object for convenience, since most analysis is on trip and vehicle
grouped = bustime.groupby(level=(0,1,2,3))
trip_veh_validation = pd.DataFrame(grouped.size(),columns=['N'])
trip_veh_validation['time_range'] = grouped['recorded_time'].max()-grouped['recorded_time'].min()
trip_veh_validation['dist_range'] = grouped['veh_dist_along_shape'].max()-grouped['veh_dist_along_shape'].min()
trip_veh_validation.head(25)

N  \
ROUTE_ID    TRIP_ID                          trip_date  vehicle_id          
MTA NYCT_B1 FB_D5-Weekday-SDon-047200_B49_15 2015-12-03 MTA NYCT_5125   4   
            FB_D5-Weekday-SDon-051000_B49_15 2015-12-03 MTA NYCT_4855   1   
                                                        MTA NYCT_5125  13   
                                                        MTA NYCT_7146   2   
            FB_D5-Weekday-SDon-051200_B49_21 2015-12-03 MTA NYCT_7146   5   
            UP_D5-Weekday-SDon-006000_B1_1   2015-12-03 MTA NYCT_4877  40   
            UP_D5-Weekday-SDon-009800_B1_1   2015-12-03 MTA NYCT_4877  56   
            UP_D5-Weekday-SDon-010000_B1_2   2015-12-03 MTA NYCT_4893  52   
            UP_D5-Weekday-SDon-013800_B1_2   2015-12-03 MTA NYCT_4893  51   
            UP_D5-Weekday-SDon-014000_B1_1   2015-12-03 MTA NYCT_4877  42   
            UP_D5-Weekday-SDon-017800_B1_1   2015-12-03 MTA NYCT_4877  53   
            UP_D5-Weekday-SDon-018000_B1_2   2015-12-03 MTA NYCT_4893  43   
            UP_D5-Weekday-SDon-021800_B1_2   2015-12-03 MTA NYCT_4893  40   
                                                        MTA NYCT_7179   1   
            UP_D5-Weekday-SDon-022000_B1_1   2015-12-03 MTA NYCT_4877  50   
            UP_D5-Weekday-SDon-025500_B1_3   2015-12-03 MTA NYCT_4990  31   
            UP_D5-Weekday-SDon-025800_B1_1   2015-12-03 MTA NYCT_4877  39   
                                                        MTA NYCT_7179   7   
                                                        MTA NYCT_7191   1   
            UP_D5-Weekday-SDon-028000_B1_4   2015-12-03 MTA NYCT_7179  38   
            UP_D5-Weekday-SDon-029400_B1_3   2015-12-03 MTA NYCT_4990  53   
                                                        MTA NYCT_7174   3   
            UP_D5-Weekday-SDon-030500_B1_5   2015-12-03 MTA NYCT_7191  35   
            UP_D5-Weekday-SDon-032400_B1_6   2015-12-03 MTA NYCT_5088  37   
            UP_D5-Weekday-SDon-033000_B1_7   2015-12-03 MTA NYCT_7174  34   

                                                                       time_range  \
ROUTE_ID    TRIP_ID                          trip_date  vehicle_id                  
MTA NYCT_B1 FB_D5-Weekday-SDon-047200_B49_15 2015-12-03 MTA NYCT_5125    00:06:54   
            FB_D5-Weekday-SDon-051000_B49_15 2015-12-03 MTA NYCT_4855    00:00:00   
                                                        MTA NYCT_5125    00:24:51   
                                                        MTA NYCT_7146    00:04:14   
            FB_D5-Weekday-SDon-051200_B49_21 2015-12-03 MTA NYCT_7146    00:06:22   
            UP_D5-Weekday-SDon-006000_B1_1   2015-12-03 MTA NYCT_4877    00:40:23   
            UP_D5-Weekday-SDon-009800_B1_1   2015-12-03 MTA NYCT_4877    00:47:00   
            UP_D5-Weekday-SDon-010000_B1_2   2015-12-03 MTA NYCT_4893    00:45:10   
            UP_D5-Weekday-SDon-013800_B1_2   2015-12-03 MTA NYCT_4893    00:41:10   
            UP_D5-Weekday-SDon-014000_B1_1   2015-12-03 MTA NYCT_4877    00:33:08   
            UP_D5-Weekday-SDon-017800_B1_1   2015-12-03 MTA NYCT_4877    00:44:01   
            UP_D5-Weekday-SDon-018000_B1_2   2015-12-03 MTA NYCT_4893    00:33:46   
            UP_D5-Weekday-SDon-021800_B1_2   2015-12-03 MTA NYCT_4893    00:31:30   
                                                        MTA NYCT_7179    00:00:00   
            UP_D5-Weekday-SDon-022000_B1_1   2015-12-03 MTA NYCT_4877    00:41:21   
            UP_D5-Weekday-SDon-025500_B1_3   2015-12-03 MTA NYCT_4990    00:24:55   
            UP_D5-Weekday-SDon-025800_B1_1   2015-12-03 MTA NYCT_4877    00:31:26   
                                                        MTA NYCT_7179    00:05:17   
                                                        MTA NYCT_7191    00:00:00   
            UP_D5-Weekday-SDon-028000_B1_4   2015-12-03 MTA NYCT_7179    00:33:21   
            UP_D5-Weekday-SDon-029400_B1_3   2015-12-03 MTA NYCT_4990    00:58:11   
                                     

## 4. Check if stop-distance values are persistent for routes, or for shape.
First look at by ROUTE_ID.  There are clearly multiple distance values reported for stops along each ROUTE_ID, with a significant number of records associated.

In [25]:
route_dist_grouped = bustime.reset_index().groupby(['ROUTE_ID','STOP_ID','distance_shape']).size()
route_dist_grouped.head(50)

ROUTE_ID     STOP_ID     distance_shape
MTA NYCT_B1  MTA_300000  0.44              258
             MTA_300002  288.75             25
                         289.31            275
             MTA_300003  426.97              1
                         427.52             23
             MTA_300004  560.27             20
             MTA_300006  845.34              2
                         845.90             37
             MTA_300007  1001.74             1
                         1002.30            29
             MTA_300008  1115.80             3
                         1116.36            42
             MTA_300009  1322.50             3
                         1323.06            66
             MTA_300010  1505.48            47
             MTA_300011  1659.30             4
                         1659.86            53
             MTA_300012  1837.44             9
                         1838.00           219
             MTA_300013  2066.28             5
                    

Now do the same for SHAPE_ID.  Only a few with multiple distance values per STOP_ID.  The first three examples have 0.00 distance for one of the duplicates.

In [26]:
shape_dist_grouped = bustime.groupby(['SHAPE_ID','STOP_ID','distance_shape']).size()
shape_dist_grouped.head(50)

SHAPE_ID      STOP_ID     distance_shape
MTA_B1000027  MTA_300788  0.00                2
              MTA_350159  834.35              7
MTA_B1000031  MTA_300788  0.00                1
                          1987.73             2
              MTA_303920  1405.31             2
              MTA_308452  11.75               1
              MTA_350217  585.11              2
MTA_B1000057  MTA_300226  4629.62           102
              MTA_300227  4743.25            51
              MTA_300788  0.00              109
              MTA_308197  1779.88            78
              MTA_350019  0.00                1
                          7226.94           219
              MTA_350060  3972.00            89
              MTA_350061  4180.54            76
              MTA_350086  6225.50            35
              MTA_350088  5707.47            27
              MTA_350091  5467.73            23
              MTA_350093  5943.34            26
              MTA_350094  5226.29            22

Examine only the SHAPE_ID-STOP_ID combinations with multiple distance values

In [63]:
shape_dist_dupes = shape_dist_grouped.groupby(level=(0,1)).size()
shape_dist_dupes[shape_dist_dupes>1]
shape_dist_dupes.name = 'duplicate_count'

Looking at the various values, it seems that one of the distance values is always 0 or very close to 0.  This could be due to errors in the Bus Time real-time inferences, or perhaps due to loop routes where the same stop is both the last and first stop.

In [66]:
dupe_summary = pd.DataFrame(shape_dist_dupes[shape_dist_dupes>1]).merge(shape_dist_grouped.reset_index(level=2),left_index=True,right_index=True,how='left')
dupe_summary.rename(columns={0:'record_count'},inplace=True)
dupe_summary

duplicate_count  distance_shape  record_count
SHAPE_ID     STOP_ID                                                  
MTA_B1000031 MTA_300788                2            0.00             1
             MTA_300788                2         1987.73             2
MTA_B1000057 MTA_350019                2            0.00             1
             MTA_350019                2         7226.94           219
MTA_B1000058 MTA_300788                2            0.00            26
             MTA_300788                2         6821.57           356
MTA_B1030109 MTA_303011                2            0.00            13
             MTA_303011                2        18660.56           312
MTA_B1030116 MTA_307840                2           73.51            20
             MTA_307840                2        11800.30           275
MTA_B1O0251  MTA_305775                2            0.27             1
             MTA_305775                2         8665.62            54
             MTA_306457                2            0.08            10
             MTA_306457                2         4757.92            91
MTA_B460218  MTA_303598                2            0.00             1
             MTA_303598                2         1516.50             7
MTA_B470076  MTA_306144                2            0.00             5
             MTA_306144                2        12670.89           260
MTA_BX130168 MTA_104065                2            0.00            36
             MTA_104065                2          943.04            37
MTA_BX230076 MTA_150038                2            0.06            12
             MTA_150038                2        10190.20           210
MTA_BXM40110 MTA_150151                2            0.00            28
             MTA_150151                2        21880.68            21
MTA_BXM60089 MTA_103294                2            0.00            11
             MTA_103294                2        19019.02           126
MTA_M660059  MTA_403497                2            0.00             1
             MTA_403497                2         2546.90           265
MTA_Q050162  MTA_553021                2            0.00             1
             MTA_553021                2         1266.41            28
...                                  ...             ...           ...
MTA_Q530417  MTA_551613                2            0.05             8
             MTA_551613                2        23430.75           696
MTA_Q640053  MTA_551843                2            0.00             2
             MTA_551843                2          484.20            95
MTA_Q650077  MTA_553139                2            0.00             3
             MTA_553139                2         3099.14            10
MTA_Q650119  MTA_553139                2            0.00            24
             MTA_553139                2        14526.86           173
MTA_Q650120  MTA_551784                2            0.07            27
             MTA_551784                2        14641.31           432
MTA_Q650121  MTA_551784                2            0.07            16
             MTA_551784                2        14641.31           111
MTA_Q660138  MTA_505066                2            0.00             6
             MTA_505066                2        11456.50           399
MTA_Q690080  MTA_550742                2            0.00             1
             MTA_550742                2         7756.49            94
MTA_Q690082  MTA_550742                2            0.00             1
             MTA_550742                2         2366.40             2
MTA_Q770467  MTA_502842                2            0.00             2
             MTA_502842                2         4138.09            27
MTA_S510082  MTA_203083                2            0.00             1
             MTA_203083                2         5282.38            17
MTA_S510083  MTA_203083                2            0.00             3
             MTA_203083                2         